In [1]:
pip install category_encoders

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\sarmen\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [2]:
pip install yellowbrick --user

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\sarmen\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [3]:
%%time

###SINGLE CELL SCRIPT###

import pandas as pd
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from yellowbrick.datasets import load_occupancy
from yellowbrick.model_selection import FeatureImportances
import pickle


df = pd.read_csv(r'E:\Desktop\Lambda_School\Group Projects\Unit 4 Sprint 4 Build Week AirBnB\listings_summary.csv')

columns = [
    'neighbourhood_group_cleansed',
    'room_type',
    'accommodates',
    'bathrooms',
    'bedrooms',
    'beds',
    'bed_type',
    'amenities',
    'price',
    'security_deposit',
    'cleaning_fee',
    'minimum_nights',
    'instant_bookable'         
]

df = df[columns]

df['bathrooms'] = df['bathrooms'].fillna(df['bathrooms'].mean()).astype(float)
df['bedrooms'] = df['bedrooms'].fillna(df['bedrooms'].mean()).astype(float)
df['beds'] = df['beds'].fillna(df['beds'].mean()).astype(float)
df['security_deposit'] = df['security_deposit'].fillna('$0.00')
df['cleaning_fee'] = df['cleaning_fee'].fillna('$0.00')

df['price'] = df['price'].str.replace('$', '').str.replace(',', '').astype(float)
df['security_deposit'] = df['security_deposit'].str.replace('$', '').str.replace(',', '').astype(float)
df['cleaning_fee'] = df['cleaning_fee'].str.replace('$', '').str.replace(',', '').astype(float)

df['neighborhood'] = df['neighbourhood_group_cleansed']
df = df.drop(columns=['neighbourhood_group_cleansed'])

df['instant_bookable'] = df['instant_bookable'].replace({'t': 1, 'f': 0})
df['neighborhood'] = df['neighborhood'].str.replace('ö', 'o')

df['amenities'] = df['amenities'].str.replace('{',"[")
df['amenities'] = df['amenities'].str.replace('}',"]")
df['amenities'] = df['amenities'].str.replace('"',"")
df['amenities'] = df['amenities'].apply(lambda x: x.lower())

df['wifi'] = df['amenities'].str.contains('wifi').astype(str)
df['wifi'] = df['wifi'].replace({'True': 1, 'False': 0})
df = df.drop(columns='amenities')

encoder = ce.OrdinalEncoder(cols = [
    'neighborhood',
    'room_type',
    'bed_type'
    ])

X = df.drop(['price'], axis=1)
y = df['price']

X_encoded = encoder.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.25)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

rfc = RandomForestClassifier(random_state=42)


rfc.fit(X_train, y_train)
preds = rfc.predict(X_test)
mean_absolute_error(preds, y_test)

params = {
#     'n_estimators': [200, 500],
    'n_estimators': [200, 400],
#     'max_features': ['auto', 'sqrt'],
    'max_features': ['auto'],
    'max_depth' : [4,5,6,7,8,15,20,25],
    'criterion' :['gini', 'entropy']
}

grid_rfc = GridSearchCV(estimator = rfc,
                        param_grid = params,
                        scoring = 'accuracy', 
                        cv = 5, 
                        verbose = 1)

grid_rfc.fit(X_train, y_train)

rfc = RandomForestClassifier(criterion='entropy', max_depth=8, n_estimators=400,
                       random_state=42)

rfc.fit(X_train, y_train)


filename = 'model.pkl'
f = open(filename, 'wb')
pickle.dump(rfc, f)

f.close()

Fitting 5 folds for each of 32 candidates, totalling 160 fits


c:\users\sarmen\appdata\local\programs\python\python38\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Wall time: 26min 17s
